# 基于Mindspore构造非对称似然损失函数

本小节主要介绍如何基于mindspore构造非对称似然损失函数。

使用 MindSpore框架中的NLLLoss 操作计算一个小批量数据的负对数似然损失（negative log-likelihood loss），并输出计算得到的损失值和加权值。

获取logits 和标签之间的负对数似然损失。

reduction=none 的 nll loss 可以描述为：


$$ \iota(x,t)=L=\{l_1,...,l_n \}^T $$

$$ l_n=-\omega _{tn}\chi _{n,tn} $$

$$ \omega _c=weight[c]\cdot1 $$


其中 x 是输入，t 是目标。w 是权重。N 是批量大小。c 属于 [0, C-1] 是类索引，其中 C 是类数。

如果 reduction 不是“none”（默认为“mean”），则


$$
 \iota(x,t)=
 \begin{cases}
 \sum_{n=1}^N \frac{1}{\sum_{n=1}^N \cdot\omega_{tn}}{l_n}, & \text{if reduction = 'mean';}\\
 \sum_{n=1}^N{l_n}, & \text{if reduction = 'sum';}
 \end{cases}
 $$

### 参数：

    reduction ( string ) – 将特定的归约方法应用于输出：'none'、'mean'、'sum'。默认值：“均值”。

### 输入：

    input (Tensor) - 输入 logits，有形状(N,C). 数据类型仅支持 float32 或 float16。

    target (Tensor) - 地面实况标签，带有形状(N). 数据类型只支持 int32。

    weight (Tensor) - 每个类的重新缩放权重，具有形状(C)并且数据类型仅支持 float32 或 float16`。


### 输出：
    由loss和total_weight组成的 2 个张量的元组。
    
    loss (Tensor) - 当reduction为none且输入为 2D 张量时，损失形状为(N,)。否则，损失是一个标量。数据类型与输入相同。
    

    total_weight (Tensor)  - total_weight是一个标量。数据类型与权重相同。

   TypeError – 如果 x 和权重数据类型不是 float16 或 float32 张量，则目标数据类型不是 int32 张量。
    

   ValueError – 如果 x 不是一维或二维张量，则目标和权重不是一维张量。当x为二维张量时，x的第一维不等于target，x的第二维不等于weight。当 x 是一维张量时，x、target 和 weight 的维度应该彼此相等。

### 导入库和函数

In [1]:
import numpy as np      #导入 NumPy 库，用于生成输入数据
import mindspore.nn as nn #导入 MindSpore 框架中的 nn 模块，用于构建神经网络
import mindspore.ops as ops #导入 MindSpore 框架中的 ops 模块，用于定义操作
from mindspore import Tensor #从 MindSpore 框架中导入 Tensor 类，用于创建张量

### 测试

In [3]:
input_tensor = Tensor(np.array([[0.5488135, 0.71518934], 
                                [0.60276335, 0.5448832],
                                [0.4236548, 0.6458941]]).astype(np.float32)) #使用 NumPy 生成一个形状为 (3, 2) 的浮点数数组，并将其转换为 MindSpore张量
target = Tensor(np.array([0, 0, 0]).astype(np.int32))                 #使用 NumPy 生成一个整数数组，并将其转换为 MindSpore 张量       
weight = Tensor(np.array([0.3834415, 0.79172504]).astype(np.float32))
nll_loss = ops.NLLLoss(reduction="mean")    #创建一个使用平均值作为减少方法的 NLLLoss 操作
loss, weight = nll_loss(input_tensor, target, weight)     #使用 NLLLoss 操作计算输入数据的损失值和加权值，并将它们保存在变量loss和 weight 中
print(loss)              # 计算得到的损失值
print(weight)            # 计算得到的加权值

-0.52507716
1.1503246
